## PyTorch中Dataset和DataLoader的基本使用

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

### 1. Dataset

In [2]:
class DatasetDemo(Dataset):
    def __init__(self, data=None):
        print("执行DatasetDemo.__init__")
        super().__init__()
        if not None:
            data = np.arange(10)
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        print("\t->获取{}号元素".format(index))
 
        try:
            return np.array([self.data[index]]), index
        except Exception as e:
            print(str(e))
            return np.array([index]), index

In [3]:
# demo = DatasetDemo()
demo = DatasetDemo(np.array([1, 2, 3, 4, 5, 6, 7, 8]))

执行DatasetDemo.__init__


In [4]:
len(demo)

10

In [5]:
demo[0]

	->获取0号元素


(array([0]), 0)

demo[9]

In [6]:
# demo[99]

### 2. DataLoader
为了进行批处理和数据加载的并行化，通常会使用`DataLoader`。    
常用的参数：
- `dataset`: 必填，数据集实例
- `batch_size`: 每个批次的样本数量， 默认是1
- `shuffle`: `True`表示每次迭代前都打乱数据顺序
- `num_workers`: 默认`0`(数据将在主进程中加载), 使用多少个子进程来并发加载数据
- `collate_fn`: 一个可选的函数，用于将赝本列表转换为小批量。默认情况下，它会堆叠张量
- `pin_memory`: 如果使用GPU，是否将张量复制到`CUDA`的固定内存中以加速数据传输，默认`False`
- `drop_last`: 如果数据集大小不能被`batch_size`整除，是否丢弃最后一个不完整的批次，默认是`False`
- `timeout`: 数据加载超时时间，单位为秒，防止加载数据的时候卡死，默认是`0`（无超时）

#### 2.1 创建DataLoader

In [7]:
def worker_init_fn(worker_id):
    # 初始化每个worker的随机种子等
    print("\t:worker_init_fn")
    np.random.seed(torch.initial_seed())

In [8]:
batch_size=2
data_loader = DataLoader(
    dataset=demo,
    batch_size=batch_size,
    num_workers=0,  # 注意这里设置的是0，如果设置了子进程来加载数据，会报错，待修复
    worker_init_fn=worker_init_fn,
    drop_last=False,
    timeout=0
)

In [9]:
type(data_loader)

torch.utils.data.dataloader.DataLoader

`DataLoader`实例化的时候`batch_size=10`,那么当for执行的时候，会执行`len(datasets) / batch_size`次

In [10]:
len(demo) / batch_size

5.0

In [11]:
# 现在我们取2次数据
count = 0
for epoch in range(1, 3):
    i = 0
    for (data, labels) in data_loader:
        i += 1
        count += 1
        print(f"epoch={epoch}, i = {i}, count={count}\tlabels:{labels}\n")
    print("")
print(f"count = {count}")

	->获取0号元素
	->获取1号元素
epoch=1, i = 1, count=1	labels:tensor([0, 1])

	->获取2号元素
	->获取3号元素
epoch=1, i = 2, count=2	labels:tensor([2, 3])

	->获取4号元素
	->获取5号元素
epoch=1, i = 3, count=3	labels:tensor([4, 5])

	->获取6号元素
	->获取7号元素
epoch=1, i = 4, count=4	labels:tensor([6, 7])

	->获取8号元素
	->获取9号元素
epoch=1, i = 5, count=5	labels:tensor([8, 9])


	->获取0号元素
	->获取1号元素
epoch=2, i = 1, count=6	labels:tensor([0, 1])

	->获取2号元素
	->获取3号元素
epoch=2, i = 2, count=7	labels:tensor([2, 3])

	->获取4号元素
	->获取5号元素
epoch=2, i = 3, count=8	labels:tensor([4, 5])

	->获取6号元素
	->获取7号元素
epoch=2, i = 4, count=9	labels:tensor([6, 7])

	->获取8号元素
	->获取9号元素
epoch=2, i = 5, count=10	labels:tensor([8, 9])


count = 10
